Welcome to the School Achievement Dashboard! Here, we'll look at various factors to educate decision-makers on how best to invest to improve student learning outcomes.

*Produced by Colin Howard and Luke Moffit for CSCI 77800 -- EthiCS, Fall 2024*

# Part 1. Poverty and Achievement

Start by running all of the cells in this notebook. Then use the dropdown below to choose whether to look at city-, state-, or national-level data.

In [1]:
#region
# @title Level selector

from ipywidgets import *
import pandas as pd
import urllib.request as rq
import plotly.express as px
import plotly.graph_objects as go
import IPython.display as display
import json

levels = {
    "New York City": "XN",
    "New York State": "NY",
    "United States": "NT"
}

level = "NT"

def set_level(choice):
    level = choice

level_selector = Dropdown(options=levels,description="Level:")
interact(set_level,choice=level_selector);

#endregion

interactive(children=(Dropdown(description='Level:', options={'New York City': 'XN', 'New York State': 'NY', '…

In [2]:
subjects = ["Mathematics","Reading","Science"]

subject = "Mathematics"

subscales = {
    "Mathematics" : "MRPCM",
    "Reading": "RRPCM",
    "Science": "SRPUV"
}

subscale = "MRPCM"

years = {
    "Mathematics": [1990, 1992, 1996, 1996, 2000, 2000, 2003, 2005, 2007, 2009, 2011, 2013, 2015, 2017, 2019, 2022][::-1],
    "Reading": [1992, 1994, 1998, 1998, 2002, 2003, 2005, 2007, 2009, 2011, 2013, 2015, 2017, 2019, 2022][::-1],
    "Science": [2009, 2015, 2019][::-1]
}

year = 2019

grades = {
    "Mathematics": [4,8],
    "Reading": [4,8,12],
    "Science": [4,8,12]
}

grade = 8

variable = "SLUNCH3"

stattype = "MN:MN,PC:P1,PC:P2,PC:P5,PC:P7,PC:P9"

stattypes = {
    "MN:MN": "Mean",
    "PC:P1": "10th Percentile",
    "PC:P2": "25th Percentile",
    "PC:P5": "50th Percentile",
    "PC:P7": "75th Percentile",
    "PC:P9": "90th Percentile"
}

marker_colors = {
    "MN:MN": px.colors.qualitative.Prism[10],
    "PC:P1": px.colors.qualitative.Prism[7],
    "PC:P2": px.colors.qualitative.Prism[5],
    "PC:P5": px.colors.qualitative.Prism[4],
    "PC:P7": px.colors.qualitative.Prism[1],
    "PC:P9": px.colors.qualitative.Prism[0]
}

def build_frame(subject="Mathematics",year=2019, level=level):
    level = level
    year = year
    subject = subject
    subscale = subscales[subject]
    response = rq.urlopen(f"https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data"
                        f"&subject={subject.lower()}"
                        f"&grade={grade}"
                        f"&subscale={subscale}"
                        f"&variable={variable}"
                        f"&jurisdiction={level}"
                        f"&stattype={stattype}"
                        f"&year={year}")
    response_string = response.read().decode('utf-8')
    try:
        df = pd.DataFrame(json.loads(response_string)['result'])
        return df
    except:
        return response_string

def build_figure(df):
    traces = []
    for stat in stattypes.keys():
        trace = go.Bar(
            x=df.loc[df['stattype']==stat]["varValueLabel"], 
            y=df.loc[df['stattype']==stat]["value"], 
            name=stattypes[stat], 
            marker={"color": marker_colors[stat]}
        )
        traces.append(trace)
    return traces

df = build_frame(subject,year)
fw = go.FigureWidget(data=build_figure(df),layout=go.Layout(barmode="group",title=f"{grade}th Grade {subject} NAEP Scores and Federal Lunch Program Eligibility, {year}, {dict(zip(levels.values(), levels.keys()))[level]}"))

def update_figure(change):
    subject = subject_widget.value
    year = year_widget.value
    level = level_selector.value
    #what happens when the API complains?
    df = build_frame(subject,year, level)
    with fw.batch_update():
        for i, stat in enumerate(stattypes):
            fw.data[i].x=df.loc[df['stattype']==stat]['varValueLabel']
            fw.data[i].y=df.loc[df['stattype']==stat]['value']
        fw.update_layout(title=f"{grade}th Grade {subject} NAEP Scores and Federal Lunch Program Eligibility, {year}, {dict(zip(levels.values(), levels.keys()))[level]}")
    
subject_widget = Dropdown(options=subjects,description="Subject:")
year_widget = Dropdown(options=years['Mathematics'],description="Year:")

def update_years(*args):
    year_widget.options = years[subject_widget.value]
level_selector.observe(update_figure, 'value')
subject_widget.observe(update_years, 'value')

subject_widget.observe(update_figure, 'value')
year_widget.observe(update_figure, 'value')

container = VBox([HBox([level_selector, subject_widget,year_widget]),fw])
container

# interact(update_figure,subject=subject_widget,year=year_widget);

AttributeError: 'str' object has no attribute 'loc'

In [19]:
import numpy
df = pd.read_csv("FundingData.csv")
#df
#different traces for each state
f = go.FigureWidget()
f.layout.title = "Per Pupil Funding by State over time"
f.layout.xaxis.title = "Year"
f.layout.yaxis.title = "Funding"
#x = range(2021, 1986, -1)
x = numpy.array(range(2021, 1986, -1))
print(x)
for state in df["State Name"].unique():
    #x is the year, y is the funding. individual bars are the state
    #y shoulud access the single row of funding for the state
    f.add_scatter(x=x,y=df.loc[df["State Name"] == state].to_numpy()[0][1:],name=state)
    #print(df.loc[df["State Name"] == state].to_numpy())
f



#line graph over time, all 50 states

[2021 2020 2019 2018 2017 2016 2015 2014 2013 2012 2011 2010 2009 2008
 2007 2006 2005 2004 2003 2002 2001 2000 1999 1998 1997 1996 1995 1994
 1993 1992 1991 1990 1989 1988 1987]


FigureWidget({
    'data': [{'name': 'ALABAMA',
              'type': 'scatter',
              'uid': '7a63c420-8539-4680-a2c6-b4ec10d33a88',
              'x': array([2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010,
                          2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998,
                          1997, 1996, 1995, 1994, 1993, 1992, 1991, 1990, 1989, 1988, 1987]),
              'y': array([11862, 10728, 10140, 10107, 9717, 9528, 9258, 9146, 9036, 8773, 8577,
                          8726, 8907, 8964, 9197, 8398, 7683, 7073, 6581, 6300, 6029, 5885, 5638,
                          5188, 4849, 4595, 4343, 4109, 3826, 3568, 3415, 3429, 3144, 3019, 2569,
                          2420], dtype=object)},
             {'name': 'ALASKA',
              'type': 'scatter',
              'uid': '565cf666-1808-4526-a093-7b4a24e85f0e',
              'x': array([2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010,
   

# Part 2. Poverty and Funding

In [24]:
#scatterplot that shows school poverty vs funding
#we already know that poverty impacts achievement, from previous graph
#now we want to show the correlation between poverty and funding 
#we are going to do this using data from NCES (?)
povertyDF = pd.read_csv("DistrictFundingData.csv")
#povertyDF
#census.gov information
censusDF = pd.read_csv("ussd22.csv")
censusDF["Agency ID - NCES Assigned [District] Latest available year"] = (censusDF["State FIPS Code"].zfill(2)) + censusDF["District ID"].zfill(5)
censusDF
#combine both FIPS ID with the distrcit ID from census data
#divide children in poverty by the total population

AttributeError: 'Series' object has no attribute 'zfill'

# Part 3. Factors Affecting Achievement